In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import tldextract
import urllib

from bs4 import BeautifulSoup

import sys
sys.path.append("/scratch/venia/web2wiki/helpers")


In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
import pyarrow.parquet as pq
import pyarrow as pa

from bs4 import BeautifulSoup

import os




os.environ['SPARK_HOME'] = "/home/veselovs/spark-3.2.1-bin-hadoop2.7"
os.environ['JAVA_HOME'] = "/home/veselovs/jdk-13.0.2"

config = pyspark.SparkConf().setAll([('spark.executor.memory', '50g'),
                                 ('spark.executor.cores', '40'),
                                 ('spark.driver.memory','50g'),
                                 ('spark.driver.maxResultSize','0'),
                                 ('spark.python.worker.memory', '5g'),
                                 ('spark.reducer.maxSizeInFlight','5g'),
                                 ('spark.rpc.message.maxSize', '1000'),
                                 ('spark.sql.autoBroadcastJoinThreshold','-1'),
                                 ('spark.local.dir', '/tmp/')])
sc = pyspark.SparkContext(conf=config)
spark = SparkSession.builder.appName('Venia').config(conf=config).getOrCreate()



22/09/06 11:30:17 WARN Utils: Your hostname, iccluster039 resolves to a loopback address: 127.0.1.1; using 10.90.38.15 instead (on interface ens786f0)
22/09/06 11:30:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/06 11:30:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/06 11:30:20 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/09/06 11:30:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/06 11:30:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/06 11:3

In [4]:
en_shares = spark.read.load("/scratch/venia/web2wiki/data/en_shares_merged2.parquet")


In [5]:
en_shares.take(10)


[Stage 3:====================================================>     (9 + 1) / 10]



[Row(title='"E" Is for Evidence', total_count=4, total_views=160, qid='Q4540279', topic_leaf='Literature', __index_level_0__=0),
 Row(title="'Till I Collapse", total_count=2, total_views=2316, qid='Q28004', topic_leaf='Music', __index_level_0__=2),
 Row(title='(164121) 2003 YT1', total_count=2, total_views=54, qid='Q15195085', topic_leaf='Space', __index_level_0__=3),
 Row(title='-s', total_count=5, total_views=40, qid='Q4545027', topic_leaf='Linguistics', __index_level_0__=5),
 Row(title='.243 Winchester', total_count=31, total_views=58788, qid='Q158704', topic_leaf='Technology', __index_level_0__=7),
 Row(title='.jm', total_count=58, total_views=1344, qid='Q41943', topic_leaf='Central_America', __index_level_0__=16),
 Row(title='.jo', total_count=56, total_views=1170, qid='Q42634', topic_leaf='Technology', __index_level_0__=23),
 Row(title='.sy', total_count=56, total_views=3248, qid='Q42778', topic_leaf='Technology', __index_level_0__=29),
 Row(title='0DFx', total_count=7, total_vie

- Curlie data
- ORES topics
- Contexts
- Number of times the domain shares the article


In [4]:
@F.udf
def extract_subdomain(x):
    if len(tldextract.extract(x).subdomain) > 1:
        y = tldextract.extract(x).subdomain +"."+tldextract.extract(x).registered_domain
    else:
        y = tldextract.extract(x).registered_domain
    return y

@F.udf
def extract_domain(x):
    y = tldextract.extract(x).registered_domain
    return y

@F.udf
def normalise_title(title):
    """ Replace _ with space, remove anchor, capitalize """
    title = title.split("/")[-1]
    title = title.split("#")[0]
    title = urllib.parse.unquote(title)
    title = title.strip()
    if len(title) > 0:
        title = title[0].upper() + title[1:]
    n_title = title.replace("_", " ")
    # if '#' in n_title:
    #     n_title = n_title.split('#')[0]
    return n_title


def clean_web_content(web_content,drop_dups = True):
    """
    Run this once... To generate the metadata file.
    """
    web_content = web_content.dropna(subset = ["url","wiki_url"])
    if drop_dups == True:
        web_content.dropDuplicates(subset=["url","wiki_url"])
    web_content=web_content.withColumn("title", normalise_title("wiki_url"))
    web_content = web_content.withColumn("subdomain", extract_subdomain("url"))
    web_content = web_content.withColumn("domain", extract_domain("url"))
    web_content = web_content.filter(F.col("title") != "")
    return web_content


# this fn is necessary for reversing the host names
@F.udf 
def reverse_reverse_host(x):
    x = x.split(".")
    x = x[-1::-1]
    x = ".".join(x)
    return x




In [6]:
web_content2 = spark.read.load("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/wiki_content.parquet")




In [8]:
wc = web_content2.limit(10000).toPandas()

In [27]:
web_content2 = pd.read_parquet("/scratch/venia/web2wiki/data/random_sample_html.parquet")


In [40]:
web_content2.sample(1000).to_csv("/scratch/venia/web2wiki/data/random_sample_html.csv")

In [35]:
# en_shares = pd.read_parquet("/scratch/venia/web2wiki/data/en_shares_merged.parquet")
en_shares2 = pd.read_parquet("/scratch/venia/web2wiki/data/en_shares_merged2.parquet")


In [38]:
en_shares.drop_duplicates("title")

,wikidb,title,domain,count,views,page_id,qid,topic_leaf,pred_qual,quartile
0,en,"""E"" Is for Evidence",freebooknotes.com,2,80,16251903,Q4540279,Literature,0.405515,1
2,en,'Till I Collapse,pungenerator.org,2,2316,25048455,Q28004,Music,0.452087,2
3,en,(164121) 2003 YT1,syfy.com,1,27,57185480,Q15195085,Space,0.684084,0
5,en,-s,uni-bielefeld.de,4,20,37003745,Q4545027,Linguistics,0.216022,0
7,en,.243 Winchester,robarguns.com,1,6532,1663105,Q158704,Technology,0.685997,2
...,...,...,...,...,...,...,...,...,...,...
12487369,en,Šefik Džaferović,portalfield.com,1,972,58707646,Q20525169,Southern_Europe,0.803012,1
12487370,en,Škoda Works,blogspot.com,3,2428,672117,Q391795,Eastern_Europe,0.763013,0
12487381,en,Ţ,unionpedia.org,1,570,2955866,Q181122,Linguistics,0.369722,0
12487383,en,Ť,thedarkmod.com,4,394,2152781,Q392901,Linguistics,0.298054,0


In [6]:
tensites = web_content2.limit(5000).toPandas()

In [20]:
def extract_comments(x):
    try:
        bs = BeautifulSoup(x, "xml")
        tags = bs.findall()
        tags_return = []
        for tag in tags:
            if tag.has_attr(re.compile("comment")):
                return True
            else:
                return False
    except: 
        return None


In [ ]:
tensites["text2"].map(extract_comments)

In [ ]:
tensites

In [10]:
tensites[tensites["text2"].str.contains("comment")]["url"].values

array(['http://blog.greenconsciousness.org/2010/03/gang-rape-threat-isnt-free-speech.html',
       'http://brooklynheightsblog.com/archives/78722',
       'http://fantasticdrivel.com/tag/dactylographic-monkeys', ...,
       'http://www.microwave-link.com/uncategorized/wr42',
       'http://www.nakedcapitalism.com/2014/11/sorry-state-progressives-economic-issues.html',
       'http://www.readingsheffield.co.uk/blue-bird-by-seiko-kinoshita'],
      dtype=object)

In [128]:
## importing files
harmonic = True
all_crawl = False
wiki_mirror = False
all_links = True


web_content = spark.read.load("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/wiki_content.parquet")
web_content = clean_web_content(web_content)

if wiki_mirror == True:
    mirrors = spark.read.load("/scratch/venia/web2wiki/data/all_wikimirror_shares.parquet")

if harmonic == True:
    
    if all_crawl:
        crawl_dump =spark.read.option("delimiter", "\t").csv(DATA_DIR + "common_crawl_dump/cc-main-2021-feb-apr-may-domain-ranks.txt.gz", header = True)
        crawl_dump = crawl_dump.withColumn("domain",reverse_reverse_host("#host_rev"))
    else:
        crawl_dump = spark.read.load("/scratch/venia/web2wiki/data/top_domains_pr.parquet")

# en_shares = spark.read.load("/scratch/venia/web2wiki/data/en_shares_merged.parquet")
# en_curlie = spark.read.load("/scratch/venia/web2wiki/data/en_curlie_metadata.parquet")
    
en_shares = pd.read_parquet("/scratch/venia/web2wiki/data/en_shares_merged2.parquet")
en_curlie = pd.read_parquet("/scratch/venia/web2wiki/data/en_curlie_metadata.parquet")
    

# url counts, use this for limiting to sites that only share one url
url_counts = spark.read.load("/scratch/venia/web2wiki/data/all_url_counts.parquet")
url_counts = url_counts.withColumn("num_wiki_on_url", F.col("count"))



In [129]:
web_content

DataFrame[url: string, a1: string, a2: string, wiki_url: string, text1: string, text2: string, title: string, subdomain: string, domain: string]

In [ ]:
web_content.filter(col(""))

In [127]:
rating = soup.find(attrs={'class':'comment'}).string.strip()

,url,a1,a2,wiki_url,text1,text2,title,subdomain,domain,lang,text,count,num_wiki_on_url
0,http://2seetheglobe.com/tag/education,p,div,http://en.wikipedia.org/wiki/Sharia#LGBT,<p>The Islamist militants imposed a strict for...,"<div class=""entry-content"">\n<div class=""wp-ca...",Sharia,2seetheglobe.com,2seetheglobe.com,en,The Islamist militants imposed a strict form o...,10,10
1,http://2transform.us/2006/04/13/diablogue-natu...,div,div,http://en.wikipedia.org/wiki/Hamlet,"<div>Today’s title is homage to the <a href=""h...","<div class=""entry-content"">\n<div>Today’s titl...",Hamlet,2transform.us,2transform.us,en,Today’s title is homage to the Tom Stoppard pl...,10,10
2,http://2transform.us/2006/04/13/diablogue-natu...,p,div,http://en.wikipedia.org/wiki/Existentialism,<p>\nI mention this not to discuss the show’s ...,"<div>Today’s title is homage to the <a href=""h...",Existentialism,2transform.us,2transform.us,en,\nI mention this not to discuss the show’s exi...,10,10
3,http://a.wholelottanothing.org/2008/10/21/how-...,p,div,http://en.wikipedia.org/wiki/List_of_countries...,<p>Hey Matt — I love this list….we’d have to r...,"<div class=""comment-content"">\n<p>Hey Matt — I...",List of countries by exports,wholelottanothing.org,wholelottanothing.org,en,Hey Matt — I love this list….we’d have to rena...,1,1
4,http://agile-aspects.michaelmahlberg.com/2018,p,div,https://en.wikipedia.org/wiki/Wiki#History,"<p>From memory, <strong>the prototype of the a...","<div class=""post-body entry-content"" id=""post-...",Wiki,agile-aspects.michaelmahlberg.com,michaelmahlberg.com,en,"From memory, the prototype of the agile coach ...",14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21725,http://www.towncharts.com/Wisconsin/Economy/Mo...,td,tr,https://en.wikipedia.org/wiki/Combined_Statist...,"<td>Cities marked with an asterisk (""*"") shoul...","<tr>\n<td>Cities marked with an asterisk (""*"")...",Combined Statistical Area,www.towncharts.com,towncharts.com,en,"Cities marked with an asterisk (""*"") should re...",3,3
21726,http://www.unexplained-mysteries.com/forum/top...,p,div,http://en.wikipedia.org/wiki/USS_Constitution,<p> The USS Constitution suffered a collision ...,"<div class=""ipsType_normal ipsType_richText ip...",USS Constitution,www.unexplained-mysteries.com,unexplained-mysteries.com,en,The USS Constitution suffered a collision atr...,1,1
21727,http://www.v-xplore.fr/category/android-app-de...,strong,p,https://en.wikipedia.org/wiki/Mobile_app,"<strong> <a aria-label=""app (opens in a new ta...","<p>An<strong> <a aria-label=""app (opens in a n...",Mobile app,www.v-xplore.fr,v-xplore.fr,en,app,8,8
21728,http://www.writtensound.com/index.php?term=hum,p,div,http://en.wikipedia.org/wiki/Aristophanes,"<p class=""description"">(Ancient Greek) frog . ...","<div class=""contentblock"">\n<p class=""entry""><...",Aristophanes,www.writtensound.com,writtensound.com,en,(Ancient Greek) frog . Features in Aristophane...,16,16


In [40]:
# # if it's a bold, emphasize, ... then we will map it to the second level location. 
# web_content.loc[web_content["a1"].isin(word_changes), "a1"] = web_content.loc[web_content["a1"].isin(word_changes), "a2"]

# # replace any hx with header
# web_content.loc[web_content["a1"].str.match("h[1-6]"), "a1"] = "header"

In [86]:
# ensure english
from langdetect import detect
from whatthelang import WhatTheLang

@F.udf
def detect_lang(x):
    wtl = WhatTheLang()
    try: 
        return wtl.predict_lang(x)
    except:
        return None
@F.udf
def extract_wiki_text(row):
    bs = BeautifulSoup(row,"xml")
    text = bs.get_text()
    if len(text)< 2:
        text = "None"
    return text
    
def sample_files(web_content, with_lang = True, prop = 0.03):
    web_content_sample = web_content.sample(prop)
    if with_lang == True:
        web_content_sample = web_content_sample.withColumn("lang", detect_lang("text1"))
        web_content_sample = web_content_sample.filter(F.col("lang") == "en")
    web_content_sample = web_content_sample.withColumn("text", extract_wiki_text(F.col("text1")))
    return web_content_sample

web_content_sample = sample_files(web_content)
web_content_sample = web_content_sample.join(url_counts, on = "url")
crawl = False
if crawl == True:
    web_content_sample = web_content_sample.join(crawl_dump, on ="domain",how="inner")

In [87]:
web_content = web_content_sample.toPandas()

In [112]:
from sampling import sample_articles, process_sample

In [118]:
sample = [15,9,8,7,6,5,5,5,5,5]

sa = sample_articles(web_content)
s1 = sa.sample_by_num_links_on_url(sample)
s1_clean = process_sample(s1)
s1_clean.to_csv("/scratch/venia/web2wiki/data/iterated_coding/4_bucketed_by_num_links.csv",index=False)

In [119]:
sample1 = sa.list_append(subset_articles).sample(frac = 1)
sample2 = sa.article_sample().sample(frac= 1)

In [120]:

web_content_sample = web_content_sample[web_content_sample["lang"] == "en"]
top_harmonic = process_sample(top_pr, harmonic = True)
top_harmonic.to_csv("/scratch/venia/web2wiki/data/iterated_coding/4_top_pr.csv",index=False)
rand_sample = process_sample(web_content_sample.sample(200))
rand_sample.to_csv("/scratch/venia/web2wiki/data/iterated_coding/4_random_articles.csv")
url_count_limited_sample = process_sample(url_count_limited_sample)
url_count_limited_sample.to_csv("/scratch/venia/web2wiki/data/iterated_coding/one_share_per_url.csv", index=False)

# for the harmonic data
harmonic_head = harmonic.head(100).sample(60)
harmonic_tail = harmonic.tail(500).sample(60)

harmonic_head = process_sample(harmonic_head, harmonic = True)
harmonic_tail = process_sample(harmonic_tail, harmonic = True)
harmonic_head.to_csv("/scratch/venia/web2wiki/data/iterated_coding/top_harmonic_sample.csv", index = False)
harmonic_tail.to_csv("/scratch/venia/web2wiki/data/iterated_coding/bottom_harmonic_sample.csv", index = False)

